In [1]:
%matplotlib widget
import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
from sstcam_simulation.utils.efficiency import CameraEfficiency, NSB_FLUX_UNIT
from sstcam_simulation.utils.sipm import SiPMSpecification
from sstcam_simulation.data import get_data
from astropy import units as u
from ipywidgets import interactive
from IPython.display import display

In [5]:
path_window = get_data("datasheet/efficiency/prod4_window.csv")
path_pde = get_data("datasheet/efficiency/prod4_pde.csv")

# Cherenkov & NSB Spectrum Efficiency

In [6]:
efficiency = CameraEfficiency(path_window=path_window, path_pde=path_pde)
cherenkov_integral = efficiency._integrate_cherenkov(
    efficiency._cherenkov_diff_flux_on_ground, 
    u.Quantity(300, u.nm), 
    u.Quantity(600, u.nm)
)
efficiency.cherenkov_scale = 100 / cherenkov_integral
scale_pde_wavelength = u.Quantity(410, u.nm)

fig, ax1 = plt.subplots()
ax2 = ax1.twinx()
ax1.plot(efficiency.wavelength, efficiency._cherenkov_diff_flux_on_ground, color='blue', alpha=0.1)
l_cherenkov = ax1.plot(efficiency.wavelength, efficiency._cherenkov_diff_flux_inside_pixel, color='blue', label="Cherenkov")
ax2.plot(efficiency.wavelength, efficiency._nsb_diff_flux_on_ground, color='red', alpha=0.1)
l_nsb = ax2.plot(efficiency.wavelength, efficiency._nsb_diff_flux_inside_pixel, color='red', label="NSB")
ax2.plot(efficiency.wavelength, efficiency._moonlight_diff_flux_on_ground, color='darkred', alpha=0.1)
l_moonlight = ax2.plot(efficiency.wavelength, efficiency._moonlight_diff_flux_inside_pixel, color='darkred', label="Moonlight")

nsb_nominal = efficiency.nominal_nsb_rate.to("MHz")
nsb_high = efficiency.high_nsb_rate.to("MHz")
moonlight_nominal = efficiency.nominal_moonlight_rate.to("MHz")
moonlight_high = efficiency.high_moonlight_rate.to("MHz")
cherenkov_pde = efficiency.effective_cherenkov_pde
t_nsb_nominal = ax1.text(0.01, 0.90, f"Nominal NSB Rate = {nsb_nominal:.2f}", transform=ax1.transAxes)
t_nsb_high = ax1.text(0.01, 0.85, f"High NSB Rate = {nsb_high:.2f}", transform=ax1.transAxes)
t_moonlight_nominal = ax1.text(0.01, 0.80, f"Nominal Moonlight Rate = {moonlight_nominal:.2f}", transform=ax1.transAxes)
t_moonlight_high = ax1.text(0.01, 0.75, f"High Moonlight Rate = {moonlight_high:.2f}", transform=ax1.transAxes)
t_cherenkov_pde = ax1.text(0.01, 0.70, f"Effective Cherenkov PDE = {cherenkov_pde:.2f}", transform=ax1.transAxes)

ax1.set_ylim(0, 0.45)
ax2.set_ylim(0, 35)
ax1.set_xlabel("Wavelength [nm]")
ax1.set_ylabel("Cherenkov photons [100 * 1 / nm]")
ax2.set_ylabel("NSB photons [ 1 / (nm m2 ns sr) ]")
fig.legend()

def slide_pde(pde):
    efficiency.scale_pde(scale_pde_wavelength, pde)
    l_nsb[0].set_ydata(efficiency._nsb_diff_flux_inside_pixel)
    l_moonlight[0].set_ydata(efficiency._moonlight_diff_flux_inside_pixel)
    l_cherenkov[0].set_ydata(efficiency._cherenkov_diff_flux_inside_pixel)
    
    nsb_nominal = efficiency.nominal_nsb_rate.to("MHz")
    nsb_high = efficiency.high_nsb_rate.to("MHz")
    moonlight_nominal = efficiency.nominal_moonlight_rate.to("MHz")
    moonlight_high = efficiency.high_moonlight_rate.to("MHz")
    cherenkov_pde = efficiency.effective_cherenkov_pde
    t_nsb_nominal.set_text(f"Nominal NSB Rate = {nsb_nominal:.2f}")
    t_nsb_high.set_text(f"High NSB Rate = {nsb_high:.2f}")
    t_moonlight_nominal.set_text(f"Nominal Moonlight Rate = {moonlight_nominal:.2f}")
    t_moonlight_high.set_text(f"High Moonlight Rate = {moonlight_high:.2f}")
    t_cherenkov_pde.set_text(f"Effective Cherenkov PDE = {cherenkov_pde:.2f}")

slider = widgets.FloatSlider(
    value=0.5,
    min=0.01,
    max=1,
    step=0.01,
    description=f'PDE @ {scale_pde_wavelength}:',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='50%'),
)
interactive(slide_pde, pde=slider)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(FloatSlider(value=0.5, description='PDE @ 410.0 nm:', layout=Layout(width='50%'), max=1.…